<a href="https://colab.research.google.com/github/NirmalenduPrakash/Document-Summarizer/blob/master/Summarizer_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [1]:
import os
os.chdir('/content/drive/My Drive/Transformer-master')

FileNotFoundError: ignored

In [18]:
!python train.py -src_data data/english.txt -trg_data data/french.txt -src_lang en -trg_lang fr -no_cuda

loading spacy tokenizers...
creating dataset and iterator... 
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
training model...
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 7, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 8, 8]) torch.Size([166, 8, 8, 64])
torch.Size([166, 8, 8, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 8, 8]) torch.Size([166, 8, 8, 64])
torch.Size([166, 8, 8, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 8, 8]) torch.Size([166, 8, 8, 64])
torch.Size([166, 8, 8, 7]) torch.Size([166, 8, 7, 64])
torch.Size([166, 8, 8, 8]) torch.Size([166, 8, 8, 64])
torch.Size([166, 8, 8, 7]) torch.Size([1

In [10]:
!python -m spacy download en
!python -m spacy download fr

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.7MB 846kB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=3a4b2457e2d6dce260e09b4f04b8e84462fa3210a866273ffdecdc74e29a739d
  Stored in directory: /tmp/pip-ephem-wheel-cache-p9oors45/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [0]:
import torch
import torch.nn as nn
import math
from torch import optim
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
# from typing import NamedTuple, List, Callable, Dict, Tuple, Optional
from collections import Counter
from random import shuffle
import numpy as np
import gzip
from torch.utils.data import Dataset, DataLoader
import pickle
from torch.nn.utils.rnn import pack_sequence
from torch.nn import functional as F

In [0]:
class Params:
  hidden_size = 512  
  shuffle=True
  learning_rate=.001
  num_epochs=10
  data_path = '/content/drive/My Drive/cnndm.gz'
  val_data_path = '/content/drive/My Drive/cnndm.val.gz'
  eps=1e-31
  batch_size=2
  model_path='/content/drive/My Drive/model.pt'
  # decoder_weights_path='/content/drive/My Drive/decoder.pt'
  losses_path='/content/drive/My Drive/val_losses.pkl'

  # Testing
  test_data_path: str = '/content/drive/My Drive/cnndm.test.gz'

In [0]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence:
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [0]:
def simple_tokenizer(text, lower=False, newline=None):
  if lower:
    text = text.lower()
  if newline is not None:  # replace newline by a token
    text = text.replace('\n', ' ' + newline + ' ')
  return text.split()

In [0]:
class Vocab(object):
  PAD = 0
  SOS = 1
  EOS = 2
  UNK = 3
  def __init__(self):
    self.word2index = {}
    self.word2count = Counter()
    self.reserved = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']
    self.index2word = self.reserved[:]
    self.embeddings = None

  def add_words(self, words):
    for word in words:
      if word not in self.word2index:
        self.word2index[word] = len(self.index2word)
        self.index2word.append(word)
    self.word2count.update(words)
    
  def __getitem__(self, item):
    if type(item) is int:
      return self.index2word[item]
    return self.word2index.get(item, self.UNK)

  def __len__(self):
    return len(self.index2word)    

In [0]:
class Dataset(nn.Module):
  def __init__(self, filename, tokenize=simple_tokenizer):
    print("Reading dataset %s..." % filename, end=' ', flush=True)
    self.filename = filename
    self.pairs = []
    self.src_len = 0
    self.tgt_len = 0
    if filename.endswith('.gz'):
      open = gzip.open
    with open(filename, 'rt', encoding='utf-8') as f:
      for i, line in enumerate(f):
        pair = line.strip().split('\t')
        if len(pair) != 2:
          print("Line %d of %s is malformed." % (i, filename))
          continue
        src = tokenize(pair[0])
        tgt = tokenize(pair[1])
        self.pairs.append((src, tgt))
    print("%d pairs." % len(self.pairs))

  def build_vocab(self):
    # word frequency
    word_counts={}
    count_words(word_counts,[src+tgr for src,tgr in self.pairs])
    vocab=Vocab()
    for word,count in word_counts.items():
        if(count>20):
            vocab.add_words([word])  
    self.vocab=vocab 
  def vectorize(self,tokens):
    return [self.vocab[token] for token in tokens]
  def unvectorize(self, indices):
    return [self.vocab[i] for i in indices]
  def __getitem__(self, index):
    return {'x':self.vectorize(self.pairs[index][0]),
            'y':self.vectorize(self.pairs[index][1]),
            'src':self.pairs[index][0],
            'trg':self.pairs[index][1],
            'x_len':len(self.pairs[index][0]),
            'y_len':len(self.pairs[index][1])}
  def __len__(self):
    return len(self.pairs)    

In [0]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 3000):
        super().__init__()
        self.d_model = d_model
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze_(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        pe = torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x += pe
        return x

In [0]:
def attention(q, k, v, d_k,mask=False):
  scores = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)
  scores = F.softmax(scores, dim=-1)  
  if mask:
    for row in range(scores.size(-2)):
      for col in range(scores.size(-1)):
        if(row>col):
          scores[:,:,row,col]=0  
    scores=scores.transpose(-2,-1)           
  output = torch.matmul(scores, v)
  return output

In [0]:
class MultiHeadAttention(nn.Module):
    def __init__(self,hidden_size,heads=8):
        super(MultiHeadAttention, self).__init__()
        self.heads=heads 
        self.d_k = hidden_size // heads
        self.hidden_size=hidden_size
        self.q_linear = nn.Linear(hidden_size, hidden_size,bias=False)
        self.v_linear = nn.Linear(hidden_size, hidden_size,bias=False)
        self.k_linear = nn.Linear(hidden_size, hidden_size,bias=False)        
    
    def forward(self, q, k, v,mask=False):        
        bs = q.size(0)
        k = self.k_linear(k).view(bs,-1,self.heads,self.d_k)
        q = self.q_linear(q).view(bs,-1,self.heads,self.d_k)
        v = self.v_linear(v).view(bs,-1,self.heads,self.d_k)

        k=k.transpose(1,2)
        q=q.transpose(1,2)
        v=v.transpose(1,2)

        scores = attention(q, k, v, self.d_k,mask)
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.hidden_size)
        return concat

In [0]:
class Norm(nn.Module):
  def __init__(self,hidden_size):
    super(Norm, self).__init__()
    self.norm=nn.LayerNorm(hidden_size,elementwise_affine=False)
  def forward(self,x,res):
    x+=self.norm(res)
    return x

In [0]:
class FeedForward(nn.Module):
  def __init__(self,hidden_size,d_ff=2048):
    super(FeedForward, self).__init__()
    self.linear_1=nn.Linear(hidden_size,d_ff)
    self.linear_2=nn.Linear(d_ff,hidden_size)
  def forward(self,x):
    output=self.linear_1(x)
    output=self.linear_2(output)
    return output

In [0]:
class Encoder(nn.Module):
  def __init__(self,hidden_size,vocab_size):
    super(Encoder, self).__init__()
    self.hidden_size=hidden_size
    self.pos=PositionalEncoder(hidden_size)
    self.embedding=nn.Embedding(vocab_size,hidden_size,padding_idx=0)
    self.attn=MultiHeadAttention(hidden_size)
    self.norm=Norm(hidden_size)
    self.ff=FeedForward(hidden_size)
  def forward(self,seq):
    embedded=self.embedding(seq)#+self.pos(seq)
    embedded+=self.pos(embedded)
    output=self.attn(embedded,embedded,embedded)
    output=self.norm(embedded,output)
    res=self.ff(output)
    output=self.norm(output,res)
    return output

In [0]:
class Decoder(nn.Module):
  def __init__(self,hidden_size,vocab_size):
    super(Decoder, self).__init__()
    self.hidden_size=hidden_size
    self.pos=PositionalEncoder(hidden_size)
    self.embedding=nn.Embedding(vocab_size,hidden_size,padding_idx=0)
    self.attn_1=MultiHeadAttention(hidden_size)
    self.attn_2=MultiHeadAttention(hidden_size)
    self.norm=Norm(hidden_size)
    self.ff=FeedForward(hidden_size)
  def forward(self,seq,enc_output):
    embedded=self.embedding(seq)
    embedded+=self.pos(embedded)
    output=self.attn_1(embedded,embedded,embedded,True)
    output=self.norm(embedded,output)
    res=self.attn_2(output,enc_output,enc_output)
    output=self.norm(output,res)
    res=self.ff(output)
    dec_output=self.norm(output,res)
    return output,dec_output,embedded     

In [0]:
class Transformer(nn.Module):
  def __init__(self,vocab,hidden_size):
    super(Transformer, self).__init__()
    self.vocab=vocab
    self.vocab_size=len(vocab)
    self.hidden_size=hidden_size
    self.encoder=Encoder(hidden_size,self.vocab_size)
    self.decoder=Decoder(hidden_size,self.vocab_size)
    self.linear=nn.Linear(hidden_size,self.vocab_size)
    self.softmax=nn.Softmax()
    self.sigmoid=nn.Sigmoid()
    self.attn=MultiHeadAttention(hidden_size) 
    # self.enc_dim=nn.Parameter(torch.tensor(1),requires_grad=False)
    # self.dec_dim=nn.Parameter(torch.tensor(1),requires_grad=False)
    # self.embd_dim=nn.Parameter(torch.tensor(1),requires_grad=False)
    self.enc_linear=nn.Linear(hidden_size,1)
    self.dec_linear=nn.Linear(hidden_size,1)
    self.embd_linear=nn.Linear(hidden_size,1)

  def forward(self,x,y,extra_vocab,expanded_word_idx):
    enc_output=self.encoder(x)
    output,dec_output,embedded=self.decoder(y,enc_output)
    gen_output=self.softmax(self.linear(dec_output))
    ptr_output=self.attn(output,enc_output,enc_output)    
    
    ptr_mean=torch.mean(ptr_output,-2).squeeze_()
    dec_output_mean=torch.mean(dec_output,-2).squeeze_()
    embedded_mean=torch.mean(embedded,-2).squeeze_()
    p_gen=self.sigmoid(self.enc_linear(ptr_mean)+self.dec_linear(dec_output_mean)+self.embd_linear(embedded_mean))
    gen_output=F.pad(gen_output,(0,len(extra_vocab)),'constant')
        
    ptr_dist=torch.matmul(output, enc_output.transpose(-2,-1))    
    ptr_dist=F.softmax(ptr_dist, dim=-1)
    gen_output.scatter_add_(2,expanded_word_idx.unsqueeze_(1).expand(-1,gen_output.size(1),-1),ptr_dist*(1-p_gen).unsqueeze_(2))
    return torch.log(gen_output+1e-5)

In [0]:
def my_collate(batch):
    max_x=np.max([item['x_len'] for item in batch])
    max_y=np.max([item['y_len'] for item in batch])
    
    x = [F.pad(torch.tensor(item['x']), (0,max_x-len(item['x'])), 'constant').tolist() for item in batch]    
    y = [F.pad(torch.tensor(item['y']), (0,max_y-len(item['y'])), 'constant').tolist() for item in batch]
    
    src=[item['src'] for item in batch]
    trg=[item['trg'] for item in batch]
    return {'x':x,'y': y,'src':src,'trg':trg,'x_len':[item['x_len'] for item in batch],'y_len':[item['y_len'] for item in batch]}

In [20]:
with torch.autograd.set_detect_anomaly(True):
  p=Params()
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  batch_size=p.batch_size
  dataset=Dataset(p.data_path)
  dataset.build_vocab()
  dataloader = DataLoader(dataset,p.batch_size,p.shuffle,collate_fn=my_collate,pin_memory=True)
  model=Transformer(dataset.vocab,p.hidden_size).to(device)
  optimizer = optim.Adam(model.parameters(), lr=p.learning_rate)
  criterion = nn.NLLLoss(ignore_index=dataset.vocab.PAD)

  for _e in range(p.num_epochs):
    for batch in tqdm(dataloader):
      extra_vocab={}
      expanded_x=[]
      expanded_y=[]
      
      x=torch.tensor(batch['x']).to(device)
      y=torch.tensor(batch['y']).to(device)
      src=batch['src']
      trg=batch['trg']

      max_src=np.max(batch['x_len'])
      for sent in src:
        vec=[]
        for token in sent:
          if(token in extra_vocab.keys()):
            vec.append(extra_vocab[token]) 
          elif(token not in dataset.vocab.word2index):
            extra_vocab[token]=len(dataset.vocab)+len(extra_vocab)
            vec.append(extra_vocab[token])
        expanded_x.append(vec + [0 for i in range(max_src-len(vec))])
      expanded_x=torch.tensor(expanded_x).to(device)  

      max_trg=np.max(batch['y_len'])
      for sent in trg:
        vec=[]
        for token in sent:
          if(token in extra_vocab.keys()):
            vec.append(extra_vocab[token]) 
          elif(token not in dataset.vocab.word2index):
            extra_vocab[token]=len(dataset.vocab)+len(extra_vocab)
            vec.append(extra_vocab[token])
        expanded_y.append(vec + [0 for i in range(max_trg-len(vec))])          
      expanded_y = torch.tensor(expanded_y).to(device)
      
      y=torch.cat((torch.tensor([dataset.vocab.SOS] * y.size(0), device=device).view(-1,1),y[:,:-1]),dim=1)
      pred=model(x,y,extra_vocab,expanded_x)
      loss=0
      for i in range(pred.size(1)):
        loss+=criterion(pred[:,i,:].squeeze_(),expanded_y[:,i])
      # loss=criterion(pred,expanded_y.unsqueeze(2))
      loss.backward()
      optimizer.step() 
      print(loss.data.item())

Reading dataset /content/drive/My Drive/cnndm.gz... 287089 pairs.


  0%|          | 0/143545 [00:03<?, ?it/s]


RuntimeError: ignored

In [2]:
a=torch.randn(2,2)
# a += torch.randn(1,2)
a.

tensor([[-0.9398,  1.5127],
        [-1.0192,  0.6220]])